In [1]:
import numpy as np 
import pandas as pd 
import fastf1

In [3]:
session = fastf1.get_session(2024, "Zandvoort", 'R')
session.load()
session.results

core           INFO 	Loading data for Dutch Grand Prix - Race [v3.5.3]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cach

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,...,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points
4,4,L NORRIS,NOR,norris,McLaren,FF8000,mclaren,Lando,Norris,Lando Norris,...,GBR,1.0,1,1.0,NaT,NaT,NaT,0 days 01:30:45.519000,Finished,26.0
1,1,M VERSTAPPEN,VER,max_verstappen,Red Bull Racing,3671C6,red_bull,Max,Verstappen,Max Verstappen,...,NED,2.0,2,2.0,NaT,NaT,NaT,0 days 00:00:22.896000,Finished,18.0
16,16,C LECLERC,LEC,leclerc,Ferrari,E80020,ferrari,Charles,Leclerc,Charles Leclerc,...,MON,3.0,3,6.0,NaT,NaT,NaT,0 days 00:00:25.439000,Finished,15.0
81,81,O PIASTRI,PIA,piastri,McLaren,FF8000,mclaren,Oscar,Piastri,Oscar Piastri,...,AUS,4.0,4,3.0,NaT,NaT,NaT,0 days 00:00:27.337000,Finished,12.0
55,55,C SAINZ,SAI,sainz,Ferrari,E80020,ferrari,Carlos,Sainz,Carlos Sainz,...,ESP,5.0,5,10.0,NaT,NaT,NaT,0 days 00:00:32.137000,Finished,10.0
11,11,S PEREZ,PER,perez,Red Bull Racing,3671C6,red_bull,Sergio,Perez,Sergio Perez,...,MEX,6.0,6,5.0,NaT,NaT,NaT,0 days 00:00:39.542000,Finished,8.0
63,63,G RUSSELL,RUS,russell,Mercedes,27F4D2,mercedes,George,Russell,George Russell,...,GBR,7.0,7,4.0,NaT,NaT,NaT,0 days 00:00:44.617000,Finished,6.0
44,44,L HAMILTON,HAM,hamilton,Mercedes,27F4D2,mercedes,Lewis,Hamilton,Lewis Hamilton,...,GBR,8.0,8,14.0,NaT,NaT,NaT,0 days 00:00:49.599000,Finished,4.0
10,10,P GASLY,GAS,gasly,Alpine,0093cc,alpine,Pierre,Gasly,Pierre Gasly,...,FRA,9.0,9,9.0,NaT,NaT,NaT,0 days 00:00:08.604000,Lapped,2.0
14,14,F ALONSO,ALO,alonso,Aston Martin,229971,aston_martin,Fernando,Alonso,Fernando Alonso,...,ESP,10.0,10,7.0,NaT,NaT,NaT,0 days 00:00:13.533000,Lapped,1.0
